# Fetching gene sequence

In [2]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "A.N.Other@example.com"

In [30]:
seq_id = '1007358254'
handle = Entrez.efetch(id=seq_id, db='nucleotide', rettype='gb', retmode='text')

In [31]:
res = SeqIO.read(handle, format='gb')

In [36]:
acc_id = res.id
acc_id

'NM_001321072.1'

In [6]:
features = {}
for f in res.features:
    features[f.type] = features.setdefault(f.type, []) + [f.__dict__]

In [7]:
row = 0
row_type = {}
for f_type in features:
    features[f_type].sort(key=lambda f: f['location']._end)
    prev_end = -1
    contiguous = 0
    row_span = 1
    for f in features[f_type]:
        start, end = f['location']._start, f['location']._end
        if start == prev_end:
            contiguous += 1
            row_span = 2
        else:
            contiguous = 0
        f['coordinates'] = [(int(start), row + (contiguous % 2)), (int(end), row + (contiguous % 2))]
        prev_end = end
    row_type[row] = f_type
    if row_span == 2:
        row_type[row+1] = f_type
    row += row_span


In [8]:
features

{'CDS': [{'coordinates': [(626, 6), (1967, 6)],
   'id': '<unknown id>',
   'location': FeatureLocation(ExactPosition(626), ExactPosition(1967), strand=1),
   'qualifiers': OrderedDict([('gene', ['CBS']),
                ('gene_synonym', ['HIP4']),
                ('EC_number', ['4.2.1.22']),
                ('note',
                 ['isoform 2 is encoded by transcript variant 5; serine sulfhydrase; beta-thionase; methylcysteine synthase']),
                ('codon_start', ['1']),
                ('product', ['cystathionine beta-synthase isoform 2']),
                ('protein_id', ['NP_001308001.1']),
                ('db_xref', ['GeneID:875', 'HGNC:HGNC:1550', 'MIM:613381']),
                ('translation',
                 ['MAKCEFFNAGGSVKDRISLRMIEDAERDGTLKPGDTIIEPTSGNTGIGLALAAAVRGYRCIIVMPEKMSSEKVDVLRALGAEIVRTPTNARFDSPESHVGVAWRLKNEIPNSHILDQYRNASNPLAHYDTTADEILQQCDGKLDMLVASVGTGGTITGIARKLKEKCPGCRIIGVDPEGSILAEPEELNQTEQTTYEVEGIGYDFIPTVLDRTVVDKWFKSNDEEAFTFARMLIAQEGLLCGGSAGSTVAVAVKAAQELQE

# Fetch variations from ClinVar

In [10]:
acc_id = 'NG_008938.1'
handle = Entrez.esearch(term='%s[Nucleotide/Protein Accession]' % acc_id, db='clinvar')

In [11]:
res = Entrez.read(handle, validate=False)
var_ids = res['IdList']
var_ids

['538705', '538704', '538703', '538702', '538701', '538700', '538699', '538698', '538697', '538696', '538695', '538694', '522181', '519594', '519593', '519592', '519591', '519590', '519589', '519588']

In [12]:
handle = Entrez.efetch(id=var_ids, db='clinvar', rettype='variation')
# https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=clinvar&id=11%2C9&rettype=variation&tool=biopython

In [13]:
import xmltodict
import hgvs.parser

clinvar_xml = handle.read()
clinvar_dict = xmltodict.parse(clinvar_xml)
hgvsparser = hgvs.parser.Parser()


In [14]:
clinvar_dict['ClinVarResult-Set']['VariationReport']

[OrderedDict([('@VariationID', '538705'),
              ('@VariationName', 'NM_000071.2(CBS):c.72G>A (p.Ala24=)'),
              ('@DateCreated', '2018-05-22'),
              ('@VariationType', 'Simple'),
              ('@DateLastUpdated', '2018-05-28'),
              ('@SubmitterCount', '1'),
              ('Species',
               OrderedDict([('@TaxonomyId', '9606'), ('#text', 'human')])),
              ('GeneList',
               OrderedDict([('@GeneCount', '1'),
                            ('Gene',
                             OrderedDict([('@GeneID', '875'),
                                          ('@Symbol', 'CBS'),
                                          ('@FullName',
                                           'cystathionine-beta-synthase'),
                                          ('@HGNCID', 'HGNC:1550'),
                                          ('@strand', '-'),
                                          ('@Type', 'submitted'),
                                         

In [20]:
variants = []
for variant in clinvar_dict['ClinVarResult-Set']['VariationReport']:
    for v in variant['Allele']['HGVSlist']['HGVS']:
        if '@AccessionVersion' in v and v['@AccessionVersion'] == acc_id:
            try:
                variants.append(hgvsparser.parse_hgvs_variant(v['#text']))
                print(v['#text'])
            except:
                print('Couldn\'t parse variant {}'.format(v['#text']))

NG_008938.1:g.8809G>A
NG_008938.1:g.15258G>A
NG_008938.1:g.14666C>T
NG_008938.1:g.20410G>A
NG_008938.1:g.14621G>A
NG_008938.1:g.16952T>A
NG_008938.1:g.22770A>G
NG_008938.1:g.16978G>A
NG_008938.1:g.8892G>A
NG_008938.1:g.8951G>C
NG_008938.1:g.14637G>C
NG_008938.1:g.20425_20426delCGinsTA
Couldn't parse variant NG_008938.1:g.17856_17857ins68
NG_008938.1:g.8843A>G
NG_008938.1:g.12327C>T
NG_008938.1:g.14618T>A
NG_008938.1:g.20412T>C
NG_008938.1:g.14626C>T
NG_008938.1:g.20455C>T
NG_008938.1:g.12403C>T


In [58]:
from mod
for variant in variants:
    print(repr(variant))

SequenceVariant(ac=NG_008938.1, type=g, posedit=8809G>A)
SequenceVariant(ac=NG_008938.1, type=g, posedit=15258G>A)
SequenceVariant(ac=NG_008938.1, type=g, posedit=14666C>T)
SequenceVariant(ac=NG_008938.1, type=g, posedit=20410G>A)
SequenceVariant(ac=NG_008938.1, type=g, posedit=14621G>A)
SequenceVariant(ac=NG_008938.1, type=g, posedit=16952T>A)
SequenceVariant(ac=NG_008938.1, type=g, posedit=22770A>G)
SequenceVariant(ac=NG_008938.1, type=g, posedit=16978G>A)
SequenceVariant(ac=NG_008938.1, type=g, posedit=8892G>A)
SequenceVariant(ac=NG_008938.1, type=g, posedit=8951G>C)
SequenceVariant(ac=NG_008938.1, type=g, posedit=14637G>C)
SequenceVariant(ac=NG_008938.1, type=g, posedit=20425_20426delinsTA)
SequenceVariant(ac=NG_008938.1, type=g, posedit=8843A>G)
SequenceVariant(ac=NG_008938.1, type=g, posedit=12327C>T)
SequenceVariant(ac=NG_008938.1, type=g, posedit=14618T>A)
SequenceVariant(ac=NG_008938.1, type=g, posedit=20412T>C)
SequenceVariant(ac=NG_008938.1, type=g, posedit=14626C>T)
Sequenc

# Fetching only features from nuccore

BaseOffsetPosition(base=-750, offset=0, datum=Datum.CDS_START, uncertain=False)

In [87]:
seq_id = 'NC_000021.9'
handle = Entrez.efetch(id=seq_id, db='nucleotide', rettype='gb', retmode='text', seq_start=43053190, seq_end=43076861)

In [88]:
res = SeqIO.parse(handle, format='gb')
seq = next(res)

In [93]:
seq.features[1].qualifiers['']

OrderedDict([('gene', ['CBS']),
             ('gene_synonym', ['HIP4']),
             ('note',
              ['cystathionine-beta-synthase; Derived by automated computational analysis using gene prediction method: BestRefSeq,Gnomon.']),
             ('db_xref', ['GeneID:875', 'HGNC:HGNC:1550', 'MIM:613381'])])

In [53]:
features = 0
for annotation in res['Bioseq-set_seq-set'][0]['Seq-entry_seq']['Bioseq']['Bioseq_annot']:
    features += len(annotation['Seq-annot_data']['Seq-annot_data_ftable'])
print(features)

15018


In [28]:
lines = (map(lambda l: l.split('\t'), res.split('\n')))
lines.pop(0)
res = Entrez.read(handle)

features = []
for line in :
    features
    

['1', '2656', 'gene']
['', '', '', 'gene', 'CBS']
['', '', '', 'gene_syn', 'HIP4']
['', '', '', 'gene_desc', 'cystathionine-beta-synthase']
['', '', '', 'db_xref', 'GeneID:875']
['', '', '', 'db_xref', 'HGNC:HGNC:1550']
['', '', '', 'db_xref', 'MIM:613381']
['1', '627', 'exon']
['', '', '', 'inference', 'alignment:Splign:2.1.0']
['615', '617', 'misc_feature']
['', '', '', 'note', 'upstream in-frame stop codon']
['627', '1967', 'CDS']
['', '', '', 'product', 'cystathionine beta-synthase isoform 2']
['', '', '', 'product', 'serine sulfhydrase']
['', '', '', 'product', 'beta-thionase']
['', '', '', 'product', 'methylcysteine synthase']
['', '', '', 'EC_number', '4.2.1.22']
['', '', '', 'protein_id', 'ref|NP_001308001.1|']
['', '', '', 'note', 'isoform 2 is encoded by transcript variant 5']
['628', '762', 'exon']
['', '', '', 'inference', 'alignment:Splign:2.1.0']
['763', '842', 'exon']
['', '', '', 'inference', 'alignment:Splign:2.1.0']
['843', '977', 'exon']
['', '', '', 'inference', 'al